In [1]:
import pandas as pd

# Read SAS file
df = pd.read_sas("/work/BMX_L.xpt", format="xport")

print(df.head())
print(df.shape)

       SEQN  BMDSTATS  BMXWT  BMIWT  BMXRECUM  BMIRECUM  BMXHEAD  BMIHEAD  \
0  130378.0       1.0   86.9    NaN       NaN       NaN      NaN      NaN   
1  130379.0       1.0  101.8    NaN       NaN       NaN      NaN      NaN   
2  130380.0       1.0   69.4    NaN       NaN       NaN      NaN      NaN   
3  130381.0       1.0   34.3    NaN       NaN       NaN      NaN      NaN   
4  130382.0       3.0   13.6    NaN       NaN       1.0      NaN      NaN   

   BMXHT  BMIHT  ...  BMXLEG  BMILEG  BMXARML  BMIARML  BMXARMC  BMIARMC  \
0  179.5    NaN  ...    42.8     NaN     42.0      NaN     35.7      NaN   
1  174.2    NaN  ...    38.5     NaN     38.7      NaN     33.7      NaN   
2  152.9    NaN  ...    38.5     NaN     35.5      NaN     36.3      NaN   
3  120.1    NaN  ...     NaN     NaN     25.4      NaN     23.4      NaN   
4    NaN    1.0  ...     NaN     NaN      NaN      1.0      NaN      1.0   

   BMXWAIST  BMIWAIST  BMXHIP  BMIHIP  
0      98.3       NaN   102.9     NaN  


In [2]:
#Check for missing values.
df.isnull().sum().sort_values(ascending=False)


BMIHEAD     8860
BMIRECUM    8842
BMXHEAD     8790
BMIHT       8726
BMIARML     8660
BMIARMC     8655
BMIWT       8515
BMIWAIST    8513
BMIHIP      8499
BMILEG      8464
BMXRECUM    8406
BMDBMIC     6368
BMXHIP      2084
BMXLEG      1525
BMXWAIST     670
BMXBMI       389
BMXHT        361
BMXARMC      298
BMXARML      292
BMXWT        106
SEQN           0
BMDSTATS       0
dtype: int64

Noted that key features like weight, height, and waist circumference have a lot of missing values.

In [3]:
# Drop rows with missing height or waist
df_clean = df[['SEQN', 'BMXHT', 'BMXWAIST']].dropna()

# Convert cm to meters (optional, depends on context)
df_clean['WHtR'] = df_clean['BMXWAIST'] / df_clean['BMXHT']


# Preview
print(df_clean[['SEQN', 'BMXWAIST', 'BMXHT', 'WHtR']].head())

       SEQN  BMXWAIST  BMXHT      WHtR
0  130378.0      98.3  179.5  0.547632
1  130379.0     114.7  174.2  0.658439
2  130380.0      93.5  152.9  0.611511
3  130381.0      70.4  120.1  0.586178
5  130386.0     106.1  173.3  0.612233


In [4]:
df_clean['WHtR_risk'] = df_clean['WHtR'] >= 0.5
print(df_clean[['SEQN', 'BMXWAIST', 'BMXHT', 'WHtR', 'WHtR_risk']].head())

       SEQN  BMXWAIST  BMXHT      WHtR  WHtR_risk
0  130378.0      98.3  179.5  0.547632       True
1  130379.0     114.7  174.2  0.658439       True
2  130380.0      93.5  152.9  0.611511       True
3  130381.0      70.4  120.1  0.586178       True
5  130386.0     106.1  173.3  0.612233       True


In [5]:
demo_df = pd.read_sas("/work/DEMO_L.xpt", format="xport")
merged_df = pd.merge(df_clean, demo_df, on='SEQN', how='inner')


print(merged_df.head())

       SEQN  BMXHT  BMXWAIST      WHtR  WHtR_risk  SDDSRVYR  RIDSTATR  \
0  130378.0  179.5      98.3  0.547632       True      12.0       2.0   
1  130379.0  174.2     114.7  0.658439       True      12.0       2.0   
2  130380.0  152.9      93.5  0.611511       True      12.0       2.0   
3  130381.0  120.1      70.4  0.586178       True      12.0       2.0   
4  130386.0  173.3     106.1  0.612233       True      12.0       2.0   

   RIAGENDR  RIDAGEYR  RIDAGEMN  ...  DMDHRGND  DMDHRAGZ  DMDHREDZ  DMDHRMAZ  \
0       1.0      43.0       NaN  ...       NaN       NaN       NaN       NaN   
1       1.0      66.0       NaN  ...       NaN       NaN       NaN       NaN   
2       2.0      44.0       NaN  ...       NaN       NaN       NaN       NaN   
3       2.0       5.0       NaN  ...       2.0       2.0       2.0       3.0   
4       1.0      34.0       NaN  ...       NaN       NaN       NaN       NaN   

   DMDHSEDZ      WTINT2YR      WTMEC2YR  SDMVSTRA  SDMVPSU  INDFMPIR  
0       N

In [6]:
missing = merged_df.isnull().sum()
missing[missing > 0].sort_values(ascending=False)

RIDAGEMN    8170
RIDEXPRG    7104
DMDHSEDZ    6839
DMDYRUSR    6784
DMDHREDZ    5889
DMDHRMAZ    5821
RIDEXAGM    5763
DMDHRGND    5760
DMDHRAGZ    5754
DMDMARTZ    2421
DMDEDUC2    2420
DMQMILIZ    2037
INDFMPIR    1030
dtype: int64

In [7]:
#Rename Gender and Age Columns for clarity.
merged_df['Gender'] = merged_df['RIAGENDR']

merged_df['Age'] = merged_df['RIDAGEYR']

In [8]:
# Drop rows missing any critical features
df_ready = merged_df.dropna(subset=['BMXHT', 'BMXWAIST', 'Age', 'Gender'])

#keep only selected columns
model_columns = ['SEQN', 'BMXHT', 'BMXWAIST', 'Age', 'Gender', 'WHtR', 'WHtR_risk']
df_model = df_ready[model_columns]


Based on the documentation: https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DEMO_L.htm

1	Male	
2	Female

In [9]:
df_model.loc[:, 'Gender'] = df_model['Gender'].map({1: 0, 2: 1})


In [10]:
print(df_model.head())
print(df_model.shape)
print(df_model.describe())


       SEQN  BMXHT  BMXWAIST   Age  Gender      WHtR  WHtR_risk
0  130378.0  179.5      98.3  43.0     0.0  0.547632       True
1  130379.0  174.2     114.7  66.0     0.0  0.658439       True
2  130380.0  152.9      93.5  44.0     1.0  0.611511       True
3  130381.0  120.1      70.4   5.0     1.0  0.586178       True
4  130386.0  173.3     106.1  34.0     0.0  0.612233       True
(8174, 7)
                SEQN        BMXHT     BMXWAIST          Age       Gender  \
count    8174.000000  8174.000000  8174.000000  8174.000000  8174.000000   
mean   136348.729141   159.773171    92.146477    40.910937     0.532665   
std      3456.151521    19.821279    22.010903    24.465777     0.498962   
min    130378.000000    79.100000    39.800000     2.000000     0.000000   
25%    133311.250000   154.500000    77.500000    17.000000     0.000000   
50%    136385.500000   163.600000    92.700000    41.500000     1.000000   
75%    139326.750000   172.200000   107.000000    63.000000     1.000000  

In [11]:
df_model['WHtR_risk'].value_counts(normalize=True)

WHtR_risk
True     0.722902
False    0.277098
Name: proportion, dtype: float64

In [16]:
df_model.to_csv('cleaned_nhanes_data', index=False)

In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import pandas as pd

In [4]:
nhanes_df = pd.read_csv('cleaned_nhanes_data')
nhanes_df.head()

,SEQN,BMXHT,BMXWAIST,Age,Gender,WHtR,WHtR_risk
0,130378.0,179.5,98.3,43.0,0.0,0.547632,True
1,130379.0,174.2,114.7,66.0,0.0,0.658439,True
2,130380.0,152.9,93.5,44.0,1.0,0.611511,True
3,130381.0,120.1,70.4,5.0,1.0,0.586178,True
4,130386.0,173.3,106.1,34.0,0.0,0.612233,True


In [7]:
# converting WHtR_risk to binary (0 = False, 1 = True)
nhanes_df['WHtR_risk'] = nhanes_df['WHtR_risk'].astype(int)
nhanes_df

,SEQN,BMXHT,BMXWAIST,Age,Gender,WHtR,WHtR_risk
0,130378.0,179.5,98.3,43.0,0.0,0.547632,1
1,130379.0,174.2,114.7,66.0,0.0,0.658439,1
2,130380.0,152.9,93.5,44.0,1.0,0.611511,1
3,130381.0,120.1,70.4,5.0,1.0,0.586178,1
4,130386.0,173.3,106.1,34.0,0.0,0.612233,1
...,...,...,...,...,...,...,...
8169,142305.0,151.4,89.0,76.0,1.0,0.587847,1
8170,142306.0,128.0,57.7,9.0,0.0,0.450781,0
8171,142308.0,173.3,98.4,50.0,0.0,0.567802,1
8172,142309.0,179.1,96.0,40.0,0.0,0.536013,1
